In [1]:
import d4rl
import gym
import torch
import numpy as np
from simple_models import GaussianPolicy
from utils import Args, rollout, ReplayMemory
import umap
import bokeh
from bokeh.io import output_notebook
import bokeh.plotting as bpl
output_notebook()

No module named 'flow'
No module named 'absl'
No module named 'carla'


Loading BokehJS ...

In [82]:
env = gym.make("hopper-expert-v0")
expert_dataset = env.get_dataset() # Size 1000000
expert_dataset = {k: expert_dataset[k] for k in expert_dataset.keys()}
expert_pool = ReplayMemory(expert_dataset["observations"], expert_dataset["actions"])
obs_dim = env.observation_space.shape[0]
act_dim = env.action_space.shape[0]

obs_dataset = expert_dataset["observations"]
act_dataset = expert_dataset["actions"]
obs_std = obs_dataset.std(0)
obs_std[obs_std == 0] = 1
normalization = obs_dataset.mean(0), obs_std, act_dataset.mean(0)
policy = GaussianPolicy(
    obs_dim,
    act_dim,
#     normalization=normalization,
#     hidden_sizes=(512, 512),
    activation="tanh",
)

policy_optim = torch.optim.Adam(policy.parameters())

### MSE not strictly better than Log Prob
### Leaky Relu not better Tanh
### l2 reg > No l2 (Over fits)
### Don't multiply last layer by 0.1
### Didn't include activation layer

In [83]:
loss_buf = []
val_ret_buf = []
for epoch in range(10):
    epoch_loss = []
    for expert_states, expert_actions in expert_pool.batches(256):
        expert_states = torch.FloatTensor(expert_states).to(policy.device)
        expert_actions = torch.FloatTensor(expert_actions).to(policy.device)
        policy_optim.zero_grad()
        mean, _, std = policy(expert_states)
        action = torch.distributions.Normal(loc=mean, scale=std).rsample()
#         loss = torch.nn.MSELoss()(action, expert_actions)
        log_prob, _, _ = policy.log_prob(expert_states, expert_actions)
        loss = -log_prob.mean()
        epoch_loss.append(loss.item())
        loss.backward()
        policy_optim.step()
    loss_buf.append(np.mean(epoch_loss))
    policy_rollout = rollout(10000, 1000, env, policy)
    val_ret = np.mean(policy_rollout["eps_ret"])
    val_ret_buf.append(val_ret)
    print(np.mean(epoch_loss), val_ret)

  0%|          | 0/10000 [00:00<?, ?it/s]

0.7297072047169842 479.5103020463281


  0%|          | 0/10000 [00:00<?, ?it/s]

0.23518728668036787 629.6384119608534


  0%|          | 0/10000 [00:00<?, ?it/s]

...

-0.4161312680554178 960.9934940020115


  0%|          | 0/10000 [00:00<?, ?it/s]

-0.46275557022368224 1083.56244183002


  0%|          | 0/10000 [00:00<?, ?it/s]

-0.5050774667774846 1222.461484796877


In [76]:
plot = bpl.figure()
plot.line(np.arange(len(loss_buf)), loss_buf)
bpl.show(plot)